In [ ]:
#This is a notebook to run a simple binary classification algorithm, using Decision Trees.

#Author: Viviana Acquaviva
#License: BSD but really should be TBD - just be nice.

import pandas as pd
import numpy as np
import sklearn.tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz

#Notes: 

#Data come from here
#from astroML.datasets import fetch_rrlyrae_combined
#X, y = fetch_rrlyrae_combined()

### Ok, now time to get real!

The previous data set was just a small/curated selection of the total, which is the one below. Let's read it in:

In [ ]:
Xbig = pd.read_csv('RRLyrae_features.txt', names = ['u-g', 'g-r', 'r-i', 'i-z'])
ybig = pd.read_csv('RRLyrae_labels.txt', header = None).astype(int)

Let's plot ALL the data, ahem!

In [ ]:
plt.scatter(Xbig['u-g'], Xbig['r-i'], \
            c = ybig.iloc[:,0].values, marker = '*', s =20, label = None, cmap = 'brg')

And let's take a look at how many positive examples (variable stars) we have.

In [ ]:
ybig.sum() 

In [ ]:
ybig.size

### Let's do some data thinking.

<br>
What is noticeable about this data set?

Do you expect a decision tree to be an optimal classifier, based on the shape of the data?

How would a classifier that puts everything in the "non-RR Lyrae" box fare on this data set?

In [ ]:
#Answers go here :) 

#Data is imbalanced!

#Probably not, with current features

#Accuracy would be (93141-483)/93141 -> pretty close to 1; recall would be 0; precision would be undefined :( 

Let's take a look at how our previous algorithm would fare on this data set.

In [ ]:
plt.scatter(Xbig['u-g'], Xbig['r-i'], \
            c = ybig.iloc[:,0].values, marker = '*', s =20, label = None, cmap = 'brg')
plt.axvline(x=0.218, linewidth =1, label = '1st split')
plt.axvline(x=0.147, linewidth =1, ls = '--', label = '2nd split')
plt.axhline(y=0.035, linewidth =1, ls = '-.', xmin = 0.53, xmax=0.65, label = '3rd split')
plt.legend();

Question: How is our old tree doing? What is it getting right and wrong?

In [ ]:
#Answers go here. 

Let's do our training process again! Here we don't have separate train and test splits so we can create them, we'll call them X_trainb, X_testb etc (for "big"). Note: we are not doing cross validation yet, which is bad!

In [ ]:
X_trainb, X_testb, y_trainb, y_testb = train_test_split(Xbig, ybig, random_state = 5)

Define and fit the model:

In [ ]:
#Fill



We can use the same plotting routine as above to visualize the new tree:

In [ ]:
dot_data = StringIO()
export_graphviz(
            modelbig, #note name change
            out_file =  dot_data,
            feature_names = list(X_trainb.columns), #here too
            class_names = ['Not var','Var'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Now let's take a look at those colors and then evaluate how the tree is doing on the training set.

In [ ]:
#Fill

Let's check the performance on the test set.

In [ ]:
#Fill

ypred = 




The accuracy is high, as expected! But is it meaningful? 

In [ ]:
#Fill

We can take a look at other metrics, and at the confusion matrix.

In [ ]:
#Define recall, estimate



In [ ]:
#Define precision, estimate



In [ ]:
#Define confusion matrix, estimate

### Recap: what have we seen so far?

Let's talk about what we should be doing to optimize this classifier, going back to the tools we mentioned.

In [ ]:
#Ideas here.

1. 

2. 

3. 




In [ ]:
modelbig

In [ ]:
# We can customize this cell as we try new models

modelX = DecisionTreeClassifier(...)
modelX.fit(X_trainb,y_trainb)
dot_data = StringIO()
export_graphviz(
            modelX,
            out_file =  dot_data,
            feature_names = list(X_trainb.columns),
            class_names = ['Not var','Var'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
#And then look at some of these to see what is happening

print(metrics.accuracy_score(y_trainb, modelX.predict(X_trainb)))
print(metrics.precision_score(y_trainb, modelX.predict(X_trainb)))
print(metrics.recall_score(y_trainb, modelX.predict(X_trainb)))

print(metrics.accuracy_score(y_testb, modelX.predict(X_testb)))
print(metrics.precision_score(y_testb, modelX.predict(X_testb)))
print(metrics.recall_score(y_testb, modelX.predict(X_testb)))

###  How to do k-fold cross validation: cross_val_score and cross_val_predict (and cross_validate)

In [ ]:
#Immporting some useful stuff

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

Let's start with a simple K fold with 10 splits.

In [ ]:
cv = KFold(n_splits = 10) 

scores = cross_val_score(modelbig, Xbig, ybig, cv = cv, scoring = ...)

print(scores)

scores.mean(), scores.std()

What happened?

In [ ]:
#Fill

We can try again.

In [ ]:
cv = KFold(n_splits = 10, ....) #Fill

scores = cross_val_score(modelbig, Xbig, ybig, cv = cv, scoring = 'recall')

print(scores)

scores.mean(), scores.std()

#### For severely imbalanced classifiers, stratification (the process of conserving the class membership distribution in each fold) might help. Let's stratify and shuffle (although TBH stratification does shuffle!)

In [ ]:
cv = StratifiedKFold(shuffle = True, n_splits = 10)

scores = cross_val_score(modelbig, Xbig, ybig, cv = cv, scoring = 'recall')

scores.mean(), scores.std()

If you are curious about Stratified K Fold vs K Fold...

In [ ]:
cv = KFold(shuffle = True, n_splits = 10)

In [ ]:
for train, test in cv.split(Xbig, ybig):
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(ybig.values.ravel()[train]), np.bincount(ybig.values.ravel()[test])))

In [ ]:
cv = StratifiedKFold(shuffle = True, n_splits = 10)

In [ ]:
for train, test in cv.split(Xbig, ybig):
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(ybig.values.ravel()[train]), np.bincount(ybig.values.ravel()[test])))

### How can we find the confusion matrix in a cross-validation scheme?

One possibility is to use the cross_val_predict utility function, which gives one realization obtained by combining the predictions on each of the test folds (should not be used for evaluation because it does not have uncertainty.)

In [ ]:
ypred = cross_val_predict(modelbig, Xbig, ybig, cv = cv) #No scoring parameter here

In [ ]:
cm = metrics.confusion_matrix(ybig, ypred)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(10,6))
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
#    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="green" if i == j else "red", fontsize = 30)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm, classes = ['Not hab', 'Hab'])

### A word about the cross_validate function

The function cross_validate is nice because it allows you to check train/test scores within the cross validation process (while so far we had only done this on a train/test split)

In [ ]:
cross_validate(modelbig, Xbig, ybig, scoring= 'recall', cv = cv, return_train_score=True)

### Learning curves

They give a view of the train/test gap and allow us to check whether more data would help.

https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

In [ ]:
#This helps visualize the training vs the test. Will help us visualize and see what would happen if we use more data or if 
#the data has plateaued. 

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy'):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    scaler = sklearn.preprocessing.StandardScaler()
    X= scaler.fit_transform(X)
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation test score")

    plt.legend(loc="best")
    return plt

In [ ]:
cv = StratifiedKFold(n_splits=10)

In [ ]:
plot_learning_curve(modelbig, 'Learning Curves', Xbig, ybig,  cv = cv, scoring = 'recall');

What is happening up there?

In [ ]:
# .....

In [ ]:
from sklearn.utils import shuffle
XbigS, ybigS = shuffle(Xbig, ybig)

In [ ]:
plot_learning_curve(modelbig, 'Learning Curves', XbigS, ybigS,  cv = cv, scoring = 'recall')

Conclusions?